In [1]:
!nvidia-smi

Fri Mar 24 17:37:03 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 531.18       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060 L...    On | 00000000:01:00.0 Off |                  N/A |
| N/A   48C    P8                3W /  N/A|    282MiB /  8188MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
FOLDER_DATASET_NAME = "roseapple_new_dataset_05122021"
LABEL_DIR = "roseapple_new_dataset_label_16032023"

In [3]:
EPOCHS = 100
trainable_layers = "heads"
save_weight_period = 1
initialize_weight = "mobile"

<h1>Import all dependencies</h1>

In [4]:
import keras
import tensorflow as tf

Using TensorFlow backend.


In [5]:
print(keras.__version__)
print(tf.__version__)

2.2.5
1.15.0


In [6]:
import os
import sys
import json
import numpy as np
import skimage.draw
import matplotlib.pyplot as plt
import pandas as pd
import time
import shutil

In [7]:
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from mrcnn import visualize
from mrcnn.visualize import display_instances

In [8]:
train_label_path = f"{LABEL_DIR}/train/train_label.json"
with open(train_label_path) as annoFile:
  train_label = json.load(annoFile)

train_set_num = len(train_label)
print(f"[TRAIN_SET_NUM] : {train_set_num}")

[TRAIN_SET_NUM] : 483


<h1>Prepare Dataset</h1>

In [9]:
from model_trainer.prepare_dataset import CustomDataset

In [10]:
with open("./trained_models/classes_map.json") as classesMap:
    classes_map = json.load(classesMap)

In [11]:
# Training dataset.
dataset_train = CustomDataset()
train_anno_dir = f"{LABEL_DIR}/train/train_label.json"
dataset_train.load_custom(FOLDER_DATASET_NAME, train_anno_dir, classes_map)
dataset_train.prepare()

{'skin': 483, 'minor': 670, 'critical': 1227}


In [12]:
# Validation dataset
dataset_val = CustomDataset()
val_anno_dir = f"{LABEL_DIR}/val/val_label.json"
dataset_val.load_custom(FOLDER_DATASET_NAME, val_anno_dir, classes_map)
dataset_val.prepare()

{'skin': 60, 'minor': 80, 'critical': 151}


<h1>Config</h1>

In [23]:
class CustomConfig(Config):
      """
          Configuration for training on the dataset.
          Derives from the base Config class and overrides some values.
      """
      # Give the configuration a recognizable name
      NAME = "object"
      BACKBONE = "mobilenetv1"
      RPN_ANCHOR_SCALES = (8 , 16, 32, 64, 128)

      # We use a GPU with 12GB memory, which can fit two images.
      # Adjust down if you use a smaller GPU.
      GPU_COUNT = 1 
      IMAGES_PER_GPU = 1

      # Number of classes (including background)
      NUM_CLASSES = len(classes_map) + 1  # Background + (desired classes)

      # Number of training steps per epoch
      STEPS_PER_EPOCH = train_set_num

      # Skip detections with < 90% confidence
      DETECTION_MIN_CONFIDENCE = 0.7

      IMAGE_MAX_DIM = 320
      IMAGE_MIN_DIM = 320

<h1>Saved Model files directory</h1>

In [14]:
from datetime import datetime, timedelta

In [15]:
train_model_time = datetime.strftime(datetime.now() + timedelta(hours=7),'%Y%m%d_%H%M%S')

In [16]:
save_trained_model_dir = f"trained_model/{train_model_time}"
save_trained_model_path = f"{save_trained_model_dir}/{CustomConfig.BACKBONE}_{train_model_time}.h5"
save_history_path = f"{save_trained_model_dir}/{CustomConfig.BACKBONE}_{train_model_time}.csv"

In [17]:
if not os.path.exists(save_trained_model_dir):
    os.makedirs(save_trained_model_dir)

<h1>Callbacks</h1>

In [18]:
callbacks_list = []

In [19]:
callbacks_list.append(
                        tf.keras.callbacks.CSVLogger(
                            save_history_path, 
                            separator=",", 
                            append=True
                        )
                    )

In [20]:
callbacks_list.append(
                        tf.keras.callbacks.EarlyStopping(
                            monitor="val_loss",
                            mode="auto"
                    )
                )

<h1>Train model</h1>

In [21]:
weights_path = initialize_weight

# Path to trained weights file
if initialize_weight in ["coco", "balloon", "mobile"]:
    weights_path = f"models/mask_rcnn_{initialize_weight}.h5"

In [ ]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", 
                          config=config,
                          model_dir=save_trained_model_dir)

model.load_weights(weights_path, 
                   by_name=True, 
                   exclude=[
                            "mrcnn_class_logits", 
                            "mrcnn_bbox_fc",
                            "mrcnn_bbox", 
                            "mrcnn_mask"
                            ]
                   )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
box_ind is deprecated, use box_indices instead


In [ ]:
model.train(
    dataset_train, 
    dataset_val,
    learning_rate=config.LEARNING_RATE,
    epochs=EPOCHS,
    layers=trainable_layers,
    custom_callbacks=callbacks_list
)

In [ ]:
history = model.keras_model.history.history